# Úkol 1 (MS Azure)

In [ ]:
%pip install azure-eventhub pandas


Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


## Inicializace

In [ ]:
import json
import random
import time
from datetime import datetime
from azure.eventhub import EventHubProducerClient, EventData
from azure.eventhub import EventHubConsumerClient
from queue import Queue
import pandas as pd

CONNECTION_STR = "*****"
EVENT_HUB_NAME = "szcasestudykotyk"

# Každý event by měl obsahovat čas vytvoření, alespoň jednu kategorickou proměnnou (například ID hry) a alespoň jednu spojitou proměnnou (například výše sázky).

def generate_event(pokus_id):
    return {
        "cas_vytvoreni": datetime.utcnow().isoformat(),
        "id_hry": random.choice(["Hra_1", "Hra_2", "Hra_3"]),
        "vyse_sazky": round(random.uniform(10, 500), 2),
        "pokus": pokus_id
    }


## Odesílání

In [ ]:
producer = EventHubProducerClient.from_connection_string(
    conn_str=CONNECTION_STR,
    eventhub_name=EVENT_HUB_NAME
)

batch = producer.create_batch()
for i in range(1000):
    ev = generate_event("test1940") #pole pro filtraci
    print(ev)
    try:
        batch.add(EventData(json.dumps(ev)))
    except ValueError:
        producer.send_batch(batch)
        batch = producer.create_batch()
        batch.add(EventData(json.dumps(ev)))

producer.send_batch(batch)
producer.close()
print("Odesláno.")


{'cas_vytvoreni': '2025-08-05T17:40:25.015315', 'id_hry': 'Hra_1', 'vyse_sazky': 99.61, 'pokus': 'test1940'}
{'cas_vytvoreni': '2025-08-05T17:40:25.016750', 'id_hry': 'Hra_1', 'vyse_sazky': 78.91, 'pokus': 'test1940'}
{'cas_vytvoreni': '2025-08-05T17:40:25.017559', 'id_hry': 'Hra_2', 'vyse_sazky': 34.92, 'pokus': 'test1940'}
{'cas_vytvoreni': '2025-08-05T17:40:25.017963', 'id_hry': 'Hra_3', 'vyse_sazky': 475.85, 'pokus': 'test1940'}
{'cas_vytvoreni': '2025-08-05T17:40:25.018335', 'id_hry': 'Hra_1', 'vyse_sazky': 40.89, 'pokus': 'test1940'}
{'cas_vytvoreni': '2025-08-05T17:40:25.018441', 'id_hry': 'Hra_1', 'vyse_sazky': 282.65, 'pokus': 'test1940'}
{'cas_vytvoreni': '2025-08-05T17:40:25.018529', 'id_hry': 'Hra_3', 'vyse_sazky': 153.68, 'pokus': 'test1940'}
{'cas_vytvoreni': '2025-08-05T17:40:25.018904', 'id_hry': 'Hra_2', 'vyse_sazky': 204.89, 'pokus': 'test1940'}
{'cas_vytvoreni': '2025-08-05T17:40:25.019033', 'id_hry': 'Hra_2', 'vyse_sazky': 19.08, 'pokus': 'test1940'}
{'cas_vytvoreni

## Přijímání

In [ ]:
events_queue = Queue()

def on_event(partition_context, event):
    if event is None:
        print("Žádný event")
        return
    try:
        data = json.loads(event.body_as_str())
        print("Přijat event:", data)
        events_queue.put(data)
    except Exception as e:
        print(f"Chyba při zpracování eventu: {e}")

client = EventHubConsumerClient.from_connection_string(
    conn_str=CONNECTION_STR,
    consumer_group="$Default",
    eventhub_name=EVENT_HUB_NAME
)

#spuštění čtení
with client:
    client.receive(
        on_event=on_event,
        starting_position="-1",
        max_wait_time=30
    )

#přenos z fronty do listu
events_list = []
while not events_queue.empty():
    events_list.append(events_queue.get())


Přijat event: {'cas_vytvoreni': '2025-08-04T12:57:47.288547', 'id_hry': 'Hra_1', 'vyse_sazky': 331.04, 'pokus': 'pokus1'}
Přijat event: {'cas_vytvoreni': '2025-08-04T12:57:47.288710', 'id_hry': 'Hra_2', 'vyse_sazky': 51.93, 'pokus': 'pokus1'}
Přijat event: {'cas_vytvoreni': '2025-08-04T12:57:47.288798', 'id_hry': 'Hra_2', 'vyse_sazky': 461.38, 'pokus': 'pokus1'}
Přijat event: {'cas_vytvoreni': '2025-08-04T12:57:47.288853', 'id_hry': 'Hra_3', 'vyse_sazky': 394.78, 'pokus': 'pokus1'}
Přijat event: {'cas_vytvoreni': '2025-08-04T12:57:47.288901', 'id_hry': 'Hra_3', 'vyse_sazky': 27.4, 'pokus': 'pokus1'}
Přijat event: {'cas_vytvoreni': '2025-08-04T12:57:47.288963', 'id_hry': 'Hra_2', 'vyse_sazky': 403.16, 'pokus': 'pokus1'}
Přijat event: {'cas_vytvoreni': '2025-08-04T12:57:47.289036', 'id_hry': 'Hra_3', 'vyse_sazky': 241.51, 'pokus': 'pokus1'}
Přijat event: {'cas_vytvoreni': '2025-08-04T12:57:47.289150', 'id_hry': 'Hra_2', 'vyse_sazky': 268.79, 'pokus': 'pokus1'}
Přijat event: {'cas_vytvore

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:132)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:132)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:129)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:129)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:715)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can

## Statistiky

In [ ]:
while not events_queue.empty():
    events_list.append(events_queue.get())

#filtrování podle pokus
filtrovane_eventy = [e for e in events_list if e.get("pokus") == "test1940"]

df = pd.DataFrame(filtrovane_eventy)

#součet sázek podle hry
metriky = df.groupby("id_hry")["vyse_sazky"].sum().reset_index()
metriky.rename(columns={"vyse_sazky": "soucet_sazek"}, inplace=True)

#ulozeni do CSV
metriky.to_csv("metriky.csv", index=False)

print("'metriky.csv'vytvořeno")
display(metriky)


'metriky.csv'vytvořeno


id_hry,soucet_sazek
Hra_1,80023.18
Hra_2,81106.82
Hra_3,86825.79
